In [51]:
import polars as pl
import plotly.express as px
import datetime as dt
import pandas as pd


df_base_peers_new = pl.read_excel("C:/Users/VHENEQUIM/Documents/Estudo/offshorePeer/base_peers_new_mapped.xlsx")
df_base_peers_old = pl.read_excel("C:/Users/VHENEQUIM/Documents/Estudo/offshorePeer/base_peers_old_mapped.xlsx")
df_base_peers_old = df_base_peers_old.filter(~pl.col("Mês").str.contains("06/2024|07/2024|08/2024|09/2024|10/2024|11/2024|12/2024"))

df_peers = pl.concat([df_base_peers_new, df_base_peers_old])
df_peers = df_peers.rename({
    "Fundo": "funds",
    "Mês": "month",
    "Pat. Liq.": "liquid_patrimony",
    "Ativo": "investment",
    "%Patrim. Liq.": "percentage",
    "R$Patrim. Liq.": "value"
}).with_columns(
    pl.col("month").str.to_datetime("%m/%Y").alias("date_for_sort")
).filter(
    (pl.col("investment_type") == "BDR") | (pl.col("investment_type") == "Invest. Ext.")
)
# Filter out months newer than 12/2024
cutoff_date = dt.datetime(2024, 12, 31)
df_peers = df_peers.filter(pl.col("date_for_sort") <= cutoff_date)

df_peers.write_excel("base_peers_mapped.xlsx")

In [52]:
df_peers.columns

['funds',
 'month',
 'liquid_patrimony',
 'investment',
 'percentage',
 'value',
 'ticker',
 'company_country',
 'area_of_work',
 'investment_type',
 'date_for_sort']

In [53]:
# Convert Excel serial dates to datetime and then format to month/year
df_rentability = pl.read_excel("C:/Users/VHENEQUIM/Documents/Estudo/offshorePeer/rentability.xlsx", has_header=True)

# Convert column names that are integers (dates) to proper date format
new_columns = []
for col in df_rentability.columns:
    if isinstance(col, int) or (isinstance(col, str) and col.isdigit()):
        # Convert Excel serial date to datetime and format as month/year
        date_int = int(col)
        # Excel's epoch starts on 1900-01-01, but we subtract 2 to account for Excel's quirks
        excel_date = dt.datetime(1900, 1, 1) + dt.timedelta(days=date_int - 2)
        new_col = excel_date.strftime("%m/%Y")
        new_columns.append(new_col)
    else:
        new_columns.append(col)

# Rename the columns
df_rentability = df_rentability.rename(dict(zip(df_rentability.columns, new_columns)))

df_rentability

Fundos,09/2022,10/2022,11/2022,12/2022,01/2023,02/2023,03/2023,04/2023,05/2023,06/2023,07/2023,08/2023,09/2023,10/2023,11/2023,12/2023,01/2024,02/2024,03/2024,04/2024,05/2024,06/2024,07/2024,08/2024,09/2024
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ALASKA""",0.044344,0.127825,-0.07437,-0.060777,0.103121,-0.121251,-0.003548,0.062208,0.117056,0.190784,0.065028,-0.053084,-0.066805,-0.061296,0.24508,0.082345,-0.078584,-0.024278,-0.054804,-0.089905,-0.060376,-0.070988,-0.030679,0.063558,-0.034183
"""MOAT""",0.026391,0.034772,-0.140915,-0.054891,-0.015429,-0.104222,-0.040475,0.059258,0.14479,0.114526,0.043829,-0.090731,-0.06015,-0.076038,0.204049,0.087173,-0.067469,0.008717,0.013596,-0.090331,-0.020752,-0.014522,-0.002233,0.093702,-0.033502
"""ARX INCOME""",0.014579,0.057602,-0.048031,-0.023245,0.024456,-0.092743,-0.030628,0.010655,0.046708,0.08851,0.037906,-0.04315,-0.013064,-0.027455,0.149521,0.052022,-0.044517,0.024613,0.030267,-0.038243,-0.019915,0.024744,0.026232,0.078628,-0.028274
"""MANTARO""",0.019092,0.100415,-0.100665,-0.02561,0.032444,-0.048816,-0.033611,0.038588,0.061602,0.090769,0.04888,-0.018148,-0.028413,-0.030629,0.116493,0.054991,-0.032999,0.010645,0.009263,-0.049625,-0.045874,-0.000689,0.017992,0.076179,-0.061333
"""VINLAND""",0.01305,0.072226,-0.077776,-0.031316,0.046576,-0.049169,-0.015866,0.029086,0.042934,0.063459,0.024427,-0.047354,-0.022217,-0.060665,0.095348,0.048241,-0.048131,0.00006,0.013241,-0.053687,-0.015995,0.01384,0.047163,0.060322,-0.017249
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""OPPORTUNITY""",0.004782,0.021838,-0.080851,-0.047057,0.047847,-0.056991,0.008567,0.046492,0.080304,0.111487,0.031343,-0.013594,-0.02446,-0.034363,0.103485,0.057501,0.010368,0.017039,0.026893,-0.00213,0.004669,0.074108,-0.010429,0.045644,-0.019739
"""LEBLON AÇÕES""",0.018428,0.099601,-0.121115,-0.047373,0.07724,-0.073824,-0.05952,0.010279,0.158413,0.097688,0.067,-0.050958,-0.051675,-0.068725,0.164248,0.068031,-0.029291,0.046071,0.06097,-0.042325,-0.016337,0.032234,0.012162,0.063609,-0.042463
"""IP""",0.000372,0.054508,-0.011962,-0.032361,0.051073,-0.051477,0.005591,0.035076,0.030299,0.075439,0.042058,-0.001229,-0.026725,-0.036935,0.113166,0.048643,0.017795,0.033532,0.005388,-0.038176,0.018778,0.008949,0.009348,0.04082,0.006624


In [54]:
# Add rentability data to df_peers using pure Polars
def add_rentability_to_peers_polars(df_peers, df_rentability):
    """
    Add rentability data from df_rentability to df_peers by matching fund name and month,
    using only Polars operations.
    
    Parameters:
    -----------
    df_peers : polars.DataFrame
        The melted dataframe with funds and months as rows
    df_rentability : polars.DataFrame
        The wide dataframe with funds as rows and months as columns
    
    Returns:
    --------
    polars.DataFrame
        The df_peers dataframe with an added 'rentability' column
    """
    # Get all column names except 'Fundos' to use as id_vars for melting
    month_columns = [col for col in df_rentability.columns if col != 'Fundos']
    
    # Melt the dataframe to get month-fund-rentability format using pure Polars
    melted_rentability = df_rentability.unpivot(
        index='Fundos',
        on=month_columns,
        variable_name='month',
        value_name='rentability'
    )
    
    # Rename to match df_peers column names
    melted_rentability = melted_rentability.rename({"Fundos": "funds"})
    
    # Join df_peers with the melted rentability data
    result = df_peers.join(
        melted_rentability,
        on=["funds", "month"],
        how="left"
    )
    
    return result

# Apply the pure Polars function
df_peers = add_rentability_to_peers_polars(df_peers, df_rentability)

# Show the first few rows to verify
# df_peers_with_rentability.select(["funds", "month", "rentability"]).head(10)
df_peers.write_excel("base_peers_mapped.xlsx")

In [59]:
def graph_per_ticker(df, investment_type, value_type = "value"):
    if investment_type == "BDR":
        df = df.filter(pl.col("investment_type") == "BDR")
    elif investment_type == "Invest. Ext.":
        df = df.filter(pl.col("investment_type") == "Invest. Ext.")
    elif investment_type == None:
        investment_type = "BDR & Invest. Ext."
    
    # Group by month and ticker, sum the values
    monthly_ticker_sum = df.group_by(["month", "ticker", "date_for_sort"]).agg(
        pl.sum(value_type).alias("total_value")
    )

    # Convert to pandas for easier manipulation with plotly
    plot_df = monthly_ticker_sum.to_pandas()

    # Sort by date to ensure chronological order
    plot_df = plot_df.sort_values('date_for_sort')
    
    # Calculate the total value per ticker across all months
    ticker_totals = plot_df.groupby('ticker')['total_value'].sum().sort_values(ascending=False)
    
    # Get ordered list of tickers for the color assignment
    ordered_tickers = ticker_totals.index.tolist()
    
    # Create stacked bar chart with plotly using the custom order for color
    fig = px.bar(
        plot_df,
        x="month",
        y="total_value",
        color="ticker",
        title=f'Investment Values by Ticker and Month - {investment_type}',
        labels={
            "month": "Month",
            "total_value": "Total Value (R$)",
            "ticker": "Ticker"
        },
        category_orders={
            "month": plot_df["month"].tolist(),  # Preserves chronological order
            "ticker": ordered_tickers,  # Use the custom order based on total values
        },
        height=600,
    )

    # Improve layout
    fig.update_layout(
        xaxis_title="Month",
        yaxis_title="Total Value (R$)",
        legend_title="Ticker",
        barmode='relative',  # Use relative instead of stack to properly handle negative values
        hovermode="closest"
    )

    # Format hover info
    fig.update_traces(
        hovertemplate="<b>%{x}</b><br>" +
                    "Ticker: %{fullData.name}<br>" +
                    "Value: R$ %{y:,.2f}<br>"
    )

    # Show the plot
    fig.show()

    return None

# graph_per_ticker(df_peers, "BDR")
# graph_per_ticker(df_peers, "BDR", "percentage")
# graph_per_ticker(df_peers, "Invest. Ext.")
# graph_per_ticker(df_peers, "Invest. Ext.", "percentage")
# graph_per_ticker(df_peers, None)
# graph_per_ticker(df_peers, None, "percentage")


In [134]:
def graph_per_ticker(df, investment_type=None, value_type="value", exclude_south_america=False):
    """
    Generate a graph showing investment values/percentages grouped by ticker over time.

    Parameters:
    -----------
    df : polars.DataFrame
        The dataframe containing the investment data
    investment_type : str or None, default=None
        Filter for specific investment type ("BDR", "Invest. Ext.", or None for both)
    value_type : str, default="value"
        The column to use for values ("value" for absolute or "percentage" for percentage)
    exclude_south_america : bool, default=False
        If True, excludes investments in companies based in South America.
    """
    filtered_df = df.clone()
    title_parts = []

    # Apply investment type filter
    if investment_type == "BDR":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "BDR")
        type_title = "BDR"
    elif investment_type == "Invest. Ext.":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "Invest. Ext.")
        type_title = "Invest. Ext."
    else:
        # Include both if None or any other value
        filtered_df = filtered_df.filter(
            (pl.col("investment_type") == "BDR") | (pl.col("investment_type") == "Invest. Ext.")
        )
        type_title = "BDR & Invest. Ext."
    title_parts.append(type_title)

    # Apply South America Filter
    if exclude_south_america:
        south_american_countries = [
            "Argentina", "Bolivia", "Brazil", "Chile", "Colombia",
            "Ecuador", "Guyana", "Paraguay", "Peru", "Suriname",
            "Uruguay", "Venezuela"
        ]
        filtered_df = filtered_df.filter(
            ~pl.col("company_country").cast(pl.Utf8).is_in(south_american_countries)
        )
        title_parts.append("(Excluding South America)")

    # Set value type label
    value_label = "Value (R$)" if value_type == "value" else "Percentage (%)"
    title_parts.append(f"({value_label})")

    graph_title = f'Investment {value_label} by Ticker and Month - {" ".join(title_parts)}'

    # Group by month and ticker, sum the values
    agg_expr = pl.sum(value_type)
    if value_type == "percentage":
        agg_expr = (agg_expr * 100) # Multiply by 100 for percentage display

    monthly_ticker_sum = filtered_df.group_by(["month", "ticker", "date_for_sort"]).agg(
        agg_expr.alias("total_value")
    )

    # Convert to pandas for easier manipulation with plotly
    plot_df = monthly_ticker_sum.to_pandas()

    # Check if data exists
    if plot_df.empty:
        print(f"No data found for the specified filters: {investment_type}, Exclude SA: {exclude_south_america}")
        return None

    # Sort by date to ensure chronological order
    plot_df = plot_df.sort_values('date_for_sort')

    # Calculate the total value per ticker across all months
    ticker_totals = plot_df.groupby('ticker')['total_value'].sum().sort_values(ascending=False)

    # Get ordered list of tickers for the color assignment
    ordered_tickers = ticker_totals.index.tolist()

    # Create stacked bar chart with plotly using the custom order for color
    fig = px.bar(
        plot_df,
        x="month",
        y="total_value",
        color="ticker",
        title=graph_title, # Use the constructed title
        labels={
            "month": "Month",
            "total_value": value_label, # Use dynamic label
            "ticker": "Ticker"
        },
        category_orders={
            "month": plot_df["month"].tolist(),  # Preserves chronological order
            "ticker": ordered_tickers,  # Use the custom order based on total values
        },
        height=600,
    )

    # Improve layout
    fig.update_layout(
        xaxis_title="Month",
        yaxis_title=value_label, # Use dynamic label
        legend_title="Ticker",
        barmode='relative',  # Use relative instead of stack to properly handle negative values
        hovermode="closest"
    )

    # Format hover info
    hover_format = "%{y:.2f}%" if value_type == "percentage" else "R$ %{y:,.2f}"
    fig.update_traces(
        hovertemplate="<b>%{x}</b><br>" +
                      "Ticker: %{fullData.name}<br>" +
                      f"Value: {hover_format}<br>"
    )

    # Return the figure instead of showing it directly
    return fig

# Example Usage:
# fig1 = graph_per_ticker(df_peers, "BDR")
# if fig1: fig1.show()

# fig2 = graph_per_ticker(df_peers, "Invest. Ext.", "percentage")
# if fig2: fig2.show()

# fig3 = graph_per_ticker(df_peers, None, "percentage", exclude_south_america=True)
# if fig3: fig3.show()

graph_per_ticker(df_peers, "BDR", "value", exclude_south_america=True).show()

In [58]:
def graph_per_fund(df, fund_name, value_type="value", investment_type=None):
    """
    Generate a graph showing the investment composition of a specific fund over time.
    
    Parameters:
    -----------
    df : polars.DataFrame
        The dataframe containing the investment data
    fund_name : str
        The name of the fund to analyze
    value_type : str, default="value"
        The column to use for values ("value" for absolute or "percentage" for percentage)
    investment_type : str or None, default=None
        Filter for specific investment type ("BDR", "Invest. Ext.", or None for both)
    """
    # Filter for the specified fund
    filtered_df = df.filter(pl.col("funds") == fund_name)
    
    # Apply investment type filter if specified
    if investment_type == "BDR":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "BDR")
        type_title = "BDR"
    elif investment_type == "Invest. Ext.":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "Invest. Ext.")
        type_title = "Invest. Ext."
    else:
        type_title = "BDR & Invest. Ext."
    
    # Set value type label for the title
    value_label = "Percentage" if value_type == "percentage" else "Value (R$)"
    
    # Group by month, ticker, and date_for_sort, sum the values
    monthly_ticker_sum = filtered_df.group_by(["month", "ticker", "date_for_sort"]).agg(
        pl.sum(value_type).alias("total_value")
    )
    
    # Convert to pandas for easier manipulation with plotly
    plot_df = monthly_ticker_sum.to_pandas()
    
    # Check if data exists
    if plot_df.empty:
        print(f"No data found for fund: {fund_name} with investment type: {type_title}")
        return None
    
    # Sort by date to ensure chronological order
    plot_df = plot_df.sort_values('date_for_sort')
    
    # Calculate the total value per ticker across all months
    ticker_totals = plot_df.groupby('ticker')['total_value'].sum().sort_values(ascending=False)
    
    # Get ordered list of tickers for the color assignment
    ordered_tickers = ticker_totals.index.tolist()
    
    # Create stacked bar chart with plotly
    fig = px.bar(
        plot_df,
        x="month",
        y="total_value",
        color="ticker",
        title=f'{fund_name} - {type_title} ({value_label})',
        labels={
            "month": "Month",
            "total_value": value_label,
            "ticker": "Ticker"
        },
        category_orders={
            "month": plot_df["month"].tolist(),  # Preserves chronological order
            "ticker": ordered_tickers,  # Use the custom order based on total values
        },
        height=600,
    )
    
    # Improve layout
    fig.update_layout(
        xaxis_title="Month",
        yaxis_title=value_label,
        legend_title="Ticker",
        barmode='relative',  # Use relative instead of stack to properly handle negative values
        hovermode="closest"
    )
    
    # Format hover info
    hover_format = "%{y:.2f}%" if value_type == "percentage" else "R$ %{y:,.2f}"
    fig.update_traces(
        hovertemplate="<b>%{x}</b><br>" +
                      "Ticker: %{fullData.name}<br>" +
                      f"Value: {hover_format}<br>"
    )
    
    # Show the plot
    fig.show()
    
    return None

# for fund in df_peers["funds"].unique():
#     graph_per_fund(df_peers, fund)

# Example usage
# graph_per_fund(df_peers, "VINLAND")
# graph_per_fund(df_peers, "Fund Name Here", "percentage")
# graph_per_fund(df_peers, "Fund Name Here", investment_type="BDR")

In [39]:
def graph_percentage_and_patrimony(df, fund_name=None, investment_type=None):
    """
    Generate a dual-axis graph showing investment percentages as bars (left axis) 
    and liquid patrimony as a line (right axis).
    
    Parameters:
    -----------
    df : polars.DataFrame
        The dataframe containing the investment data
    fund_name : str or None, default=None
        The name of the fund to analyze (None for all funds)
    investment_type : str or None, default=None
        Filter for specific investment type ("BDR", "Invest. Ext.", or None for both)
    """
    # Apply filters
    filtered_df = df.clone()
    
    if fund_name is not None:
        filtered_df = filtered_df.filter(pl.col("funds") == fund_name)
        title_prefix = fund_name
    else:
        title_prefix = "All Funds"
    
    if investment_type == "BDR":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "BDR")
        type_title = "BDR"
    elif investment_type == "Invest. Ext.":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "Invest. Ext.")
        type_title = "Invest. Ext."
    else:
        type_title = "BDR & Invest. Ext."
    
    # Group by month, ticker for percentages
    monthly_ticker_pct = filtered_df.group_by(["month", "ticker", "date_for_sort"]).agg(
        pl.sum("percentage").alias("total_percentage")
    )
    
    # Group by month for liquid patrimony
    monthly_patrimony = filtered_df.group_by(["month", "date_for_sort"]).agg(
        pl.mean("liquid_patrimony").alias("avg_patrimony")
    ).sort("date_for_sort")
    
    # Convert to pandas
    plot_df_pct = monthly_ticker_pct.to_pandas()
    plot_df_patrimony = monthly_patrimony.to_pandas()
    
    # Check if data exists
    if plot_df_pct.empty:
        print(f"No data found for: {title_prefix} with investment type: {type_title}")
        return None
    
    # Sort by date
    plot_df_pct = plot_df_pct.sort_values('date_for_sort')
    
    # Calculate the total percentage per ticker across all months
    ticker_totals = plot_df_pct.groupby('ticker')['total_percentage'].sum().sort_values(ascending=False)
    ordered_tickers = ticker_totals.index.tolist()
    
    # Create the bar chart for percentages
    fig = px.bar(
        plot_df_pct,
        x="month",
        y="total_percentage",
        color="ticker",
        title=f'{title_prefix} - {type_title}: Investment Percentage and Liquid Patrimony',
        labels={
            "month": "Month",
            "total_percentage": "Percentage (%)",
            "ticker": "Ticker"
        },
        category_orders={
            "month": plot_df_pct["month"].tolist(),
            "ticker": ordered_tickers,
        },
        height=600,
    )
    
    # Add the line chart for liquid patrimony
    # Instead of adding traces one by one, add a single trace with all points
    fig.add_trace(
        dict(
            type='scatter',
            x=plot_df_patrimony['month'],
            y=plot_df_patrimony['avg_patrimony'],
            name='Liquid Patrimony',
            mode='lines+markers',
            line=dict(color='black', width=3),
            marker=dict(size=8, color='black'),
            yaxis='y2'  # Use secondary y-axis
        )
    )
    
    # Set up the secondary y-axis for liquid patrimony
    fig.update_layout(
        xaxis_title="Month",
        yaxis_title="Percentage (%)",
        yaxis2=dict(
            title="Liquid Patrimony (R$)",
            titlefont=dict(color="black"),
            tickfont=dict(color="black"),
            anchor="x",
            overlaying="y",
            side="right"
        ),
        legend_title="Ticker",
        barmode='relative',
        hovermode="closest"
    )
    
    # Format hover info
    fig.update_traces(
        selector=dict(type='bar'),
        hovertemplate="<b>%{x}</b><br>" +
                      "Ticker: %{fullData.name}<br>" +
                      "Percentage: %{y:.2f}%<br>"
    )
    
    # Add hover template for the line
    fig.update_traces(
        selector=dict(name='Liquid Patrimony'),
        hovertemplate="<b>%{x}</b><br>" +
                      "Liquid Patrimony: R$ %{y:,.2f}<br>"
    )
    
    # Show the plot
    fig.show()
    
    return None

# Example usage:
# For a specific fund
graph_percentage_and_patrimony(df_peers, "IP")

In [57]:
def graph_percentage_and_rentability(df, fund_name=None, investment_type=None):
    """
    Generate a dual-axis graph showing investment percentages as bars (left axis) 
    and fund rentability as a line (right axis).
    
    Parameters:
    -----------
    df : polars.DataFrame
        The dataframe containing the investment data
    fund_name : str or None, default=None
        The name of the fund to analyze (None for all funds)
    investment_type : str or None, default=None
        Filter for specific investment type ("BDR", "Invest. Ext.", or None for both)
    """
    # Apply filters
    filtered_df = df.clone()
    
    if fund_name is not None:
        filtered_df = filtered_df.filter(pl.col("funds") == fund_name)
        title_prefix = fund_name
    else:
        title_prefix = "All Funds"
    
    if investment_type == "BDR":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "BDR")
        type_title = "BDR"
    elif investment_type == "Invest. Ext.":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "Invest. Ext.")
        type_title = "Invest. Ext."
    else:
        type_title = "BDR & Invest. Ext."
    
    # Group by month, ticker for percentages
    monthly_ticker_pct = filtered_df.group_by(["month", "ticker", "date_for_sort"]).agg(
        pl.sum("percentage").alias("total_percentage")
    )
    
    # Group by month for rentability
    monthly_rentability = filtered_df.group_by(["month", "date_for_sort"]).agg(
        (pl.mean("rentability")*100).alias("avg_rentability")
    ).sort("date_for_sort")
    
    # Convert to pandas
    plot_df_pct = monthly_ticker_pct.to_pandas()
    plot_df_rentability = monthly_rentability.to_pandas()
    
    # Check if data exists
    if plot_df_pct.empty:
        print(f"No data found for: {title_prefix} with investment type: {type_title}")
        return None
    
    # Sort by date
    plot_df_pct = plot_df_pct.sort_values('date_for_sort')
    
    # Calculate the total percentage per ticker across all months
    ticker_totals = plot_df_pct.groupby('ticker')['total_percentage'].sum().sort_values(ascending=False)
    ordered_tickers = ticker_totals.index.tolist()
    
    # Create the bar chart for percentages
    fig = px.bar(
        plot_df_pct,
        x="month",
        y="total_percentage",
        color="ticker",
        title=f'{title_prefix} - {type_title}: Investment Percentage and Rentability',
        labels={
            "month": "Month",
            "total_percentage": "Percentage (%)",
            "ticker": "Ticker"
        },
        category_orders={
            "month": plot_df_pct["month"].tolist(),
            "ticker": ordered_tickers,
        },
        height=600,
    )
    
    # Add the line chart for rentability
    fig.add_trace(
        dict(
            type='scatter',
            x=plot_df_rentability['month'],
            y=plot_df_rentability['avg_rentability'],
            name='Rentability',
            mode='lines+markers',
            line=dict(color='black', width=3),
            marker=dict(size=8, color='black'),
            yaxis='y2'  # Use secondary y-axis
        )
    )
    
    # Set up the secondary y-axis for rentability
    fig.update_layout(
        xaxis_title="Month",
        yaxis_title="Percentage (%)",
        yaxis2=dict(
            title="Rentability (%)",
            titlefont=dict(color="black"),
            tickfont=dict(color="black"),
            anchor="x",
            overlaying="y",
            side="right"
        ),
        legend_title="Ticker",
        barmode='relative',
        hovermode="closest"
    )
    
    # Format hover info
    fig.update_traces(
        selector=dict(type='bar'),
        hovertemplate="<b>%{x}</b><br>" +
                      "Ticker: %{fullData.name}<br>" +
                      "Percentage: %{y:.2f}%<br>"
    )
    
    # Add hover template for the line
    fig.update_traces(
        selector=dict(name='Rentability'),
        hovertemplate="<b>%{x}</b><br>" +
                      "Rentability: %{y:.2f}%<br>"
    )
    
    # Show the plot
    fig.show()
    
    return None

# Example usage:
# For a specific fund
graph_percentage_and_rentability(df_peers, "IP")

In [ ]:
import polars as pl
import pandas as pd
import plotly.express as px
import numpy as np

def analyze_performance_vs_investment_percentage(
    df,
    investment_type=None,
    exclude_south_america=False
):
    """
    Analyzes the correlation between average fund performance (rentability)
    and the average percentage allocated to specified investment types,
    with an option to exclude South American companies.

    Parameters:
    -----------
    df : polars.DataFrame
        The dataframe containing the investment data, including 'funds',
        'month', 'percentage', 'rentability', 'investment_type', and 'company_country' columns.
    investment_type : str or None, default=None
        Filter for specific investment type:
        - "BDR": Only include BDR investments.
        - "Invest. Ext.": Only include Invest. Ext. investments.
        - None (default): Include both BDR and Invest. Ext. investments.
    exclude_south_america : bool, default=False
        If True, excludes investments in companies based in South America.
    """
    # --- 1. Filter Data ---
    filtered_df = df.clone()
    title_parts = []

    # Apply Investment Type Filter
    if investment_type == "BDR":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "BDR")
        type_title = "BDR"
    elif investment_type == "Invest. Ext.":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "Invest. Ext.")
        type_title = "Invest. Ext."
    else:
        # Use both BDR and Invest. Ext. if None is specified
        filtered_df = filtered_df.filter(
            (pl.col("investment_type") == "BDR") | (pl.col("investment_type") == "Invest. Ext.")
        )
        type_title = "BDR & Invest. Ext."
    title_parts.append(type_title)

    # Apply South America Filter
    if exclude_south_america:
        south_american_countries = [
            "Argentina", "Bolivia", "Brazil", "Chile", "Colombia",
            "Ecuador", "Guyana", "Paraguay", "Peru", "Suriname",
            "Uruguay", "Venezuela"
            # Add French Guiana if needed, though it's technically an overseas department of France
        ]
        # Ensure the country column is string type for comparison
        filtered_df = filtered_df.filter(
            ~pl.col("company_country").cast(pl.Utf8).is_in(south_american_countries)
        )
        title_parts.append("(Excluding South America)")

    analysis_title = " ".join(title_parts)

    # --- 2. Calculate Monthly Metrics per Fund ---
    # Group by fund and month to get total percentage and unique rentability
    # Ensure rentability is numeric before aggregation
    monthly_fund_metrics = filtered_df.group_by(["funds", "month"]).agg(
        pl.sum("percentage").alias("total_monthly_percentage"),
        pl.first("rentability").cast(pl.Float64).alias("monthly_rentability") # Cast to float
    )

    # --- 3. Aggregate Data per Fund ---
    # Calculate average percentage and average rentability for each fund over time
    fund_summary = monthly_fund_metrics.group_by("funds").agg(
        (pl.mean("total_monthly_percentage")*100).alias("avg_investment_percentage"),
        pl.mean("monthly_rentability").alias("avg_rentability")
    ).drop_nulls() # Remove funds with missing data for either metric

    # Convert to Pandas for correlation and plotting
    fund_summary_pd = fund_summary.to_pandas()

    if fund_summary_pd.shape[0] < 2:
        print(f"Not enough data points ({fund_summary_pd.shape[0]}) to calculate correlation for {analysis_title}.")
        return None, None

    # --- 4. Correlation Analysis ---
    # Ensure columns are numeric before calculating correlation
    fund_summary_pd['avg_investment_percentage'] = pd.to_numeric(fund_summary_pd['avg_investment_percentage'], errors='coerce')
    fund_summary_pd['avg_rentability'] = pd.to_numeric(fund_summary_pd['avg_rentability'], errors='coerce')
    fund_summary_pd = fund_summary_pd.dropna(subset=['avg_investment_percentage', 'avg_rentability'])

    if fund_summary_pd.shape[0] < 2:
        print(f"Not enough numeric data points ({fund_summary_pd.shape[0]}) after cleaning for {analysis_title}.")
        return None, None

    correlation = fund_summary_pd['avg_investment_percentage'].corr(fund_summary_pd['avg_rentability'])

    print(f"Analysis for Investment Type: {analysis_title}")
    print(f"Correlation between Average Investment Percentage and Average Rentability: {correlation:.4f}")

    # --- 5. Visualization ---
    plot_title = f'Fund Performance vs. Avg. Investment Percentage ({analysis_title})<br>Correlation: {correlation:.4f}'
    fig = px.scatter(
        fund_summary_pd,
        x="avg_investment_percentage",
        y="avg_rentability",
        title=plot_title,
        labels={
            "avg_investment_percentage": f"Average Monthly Investment Percentage ({type_title}) (%)",
            "avg_rentability": "Average Monthly Rentability (%)"
        },
        hover_name="funds", # Show fund name on hover
        trendline="ols", # Add a linear regression trendline
        trendline_color_override="red"
    )

    fig.update_traces(
        marker=dict(size=10),
        hovertemplate="<b>%{hovertext}</b><br>" +
                      "Avg. Investment Pct: %{x:.2f}%<br>" +
                      "Avg. Rentability: %{y:.4f}%<extra></extra>"
    )

    fig.update_layout(height=600)

    return fig
# --- Example Usage ---
# Make sure df_peers is loaded and contains the necessary columns

# Analyze for BDR & Invest. Ext. combined (default), including South America (default)
# corr_both_all, summary_both_all = analyze_performance_vs_investment_percentage(df_peers)

# Analyze for BDR only, including South America
# corr_bdr_all, summary_bdr_all = analyze_performance_vs_investment_percentage(df_peers, investment_type="BDR")

# Analyze for Invest. Ext. only, excluding South America
# corr_ext_no_sa, summary_ext_no_sa = analyze_performance_vs_investment_percentage(df_peers, investment_type="Invest. Ext.", exclude_south_america=True)

# Analyze for BDR & Invest. Ext. combined, excluding South America
analyze_performance_vs_investment_percentage(df_peers, exclude_south_america=True)

Analysis for Investment Type: BDR & Invest. Ext. (Excluding South America)
Correlation between Average Investment Percentage and Average Rentability: 0.1828


In [102]:
def graph_percentage_and_patrimony(df, fund_name=None, investment_type=None, exclude_south_america=False):
    """
    Generate a dual-axis graph showing investment percentages as bars (left axis) 
    and liquid patrimony as a line (right axis).
    
    Parameters:
    -----------
    df : polars.DataFrame
        The dataframe containing the investment data
    fund_name : str or None, default=None
        The name of the fund to analyze (None for all funds)
    investment_type : str or None, default=None
        Filter for specific investment type ("BDR", "Invest. Ext.", or None for both)
    exclude_south_america : bool, default=False
        If True, excludes investments in companies based in South America.
    """
    # Apply filters
    filtered_df = df.clone()
    title_parts = []
    # Group by month for liquid patrimony
    monthly_patrimony = filtered_df.clone().filter(pl.col("funds") == fund_name).group_by(["month", "date_for_sort"]).agg(
        pl.first("liquid_patrimony").alias("avg_patrimony")
    ).sort("date_for_sort")
    
    if fund_name is not None:
        filtered_df = filtered_df.filter(pl.col("funds") == fund_name)
        title_prefix = fund_name
    else:
        title_prefix = "All Funds"
    title_parts.append(title_prefix)
    
    if investment_type == "BDR":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "BDR")
        type_title = "BDR"
    elif investment_type == "Invest. Ext.":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "Invest. Ext.")
        type_title = "Invest. Ext."
    else:
        type_title = "BDR & Invest. Ext."
    title_parts.append(type_title)

    # Apply South America Filter
    if exclude_south_america:
        south_american_countries = [
            "Argentina", "Bolivia", "Brazil", "Chile", "Colombia",
            "Ecuador", "Guyana", "Paraguay", "Peru", "Suriname",
            "Uruguay", "Venezuela"
        ]
        # Ensure the country column is string type for comparison
        filtered_df = filtered_df.filter(
            ~pl.col("company_country").cast(pl.Utf8).is_in(south_american_countries)
        )
        title_parts.append("(Excluding South America)")

    graph_title = f'{title_parts[0]} - {title_parts[1]}: Investment Percentage and Liquid Patrimony'
    if len(title_parts) > 2:
        graph_title += f' {title_parts[2]}'
    
    # Group by month, ticker for percentages
    monthly_ticker_pct = filtered_df.group_by(["month", "ticker", "date_for_sort"]).agg(
        pl.sum("percentage").alias("total_percentage")
    )

    # Convert to pandas
    plot_df_pct = monthly_ticker_pct.to_pandas()
    plot_df_patrimony = monthly_patrimony.to_pandas()
    
    # Check if data exists
    if plot_df_pct.empty:
        print(f"No data found for: {title_prefix} with investment type: {type_title}")
        return None
    
    # Sort by date
    plot_df_pct = plot_df_pct.sort_values('date_for_sort')
    
    # Calculate the total percentage per ticker across all months
    ticker_totals = plot_df_pct.groupby('ticker')['total_percentage'].sum().sort_values(ascending=False)
    ordered_tickers = ticker_totals.index.tolist()
    
    # Create the bar chart for percentages
    fig = px.bar(
        plot_df_pct,
        x="month",
        y="total_percentage",
        color="ticker",
        title=graph_title,
        labels={
            "month": "Month",
            "total_percentage": "Percentage (%)",
            "ticker": "Ticker"
        },
        category_orders={
            "month": plot_df_pct["month"].tolist(),
            "ticker": ordered_tickers,
        },
        height=600,
    )
    
    # Add the line chart for liquid patrimony
    # Instead of adding traces one by one, add a single trace with all points
    fig.add_trace(
        dict(
            type='scatter',
            x=plot_df_patrimony['month'],
            y=plot_df_patrimony['avg_patrimony'],
            name='Liquid Patrimony',
            mode='lines+markers',
            line=dict(color='black', width=3),
            marker=dict(size=8, color='black'),
            yaxis='y2'  # Use secondary y-axis
        )
    )
    
    # Set up the secondary y-axis for liquid patrimony
    fig.update_layout(
        xaxis_title="Month",
        yaxis_title="Percentage (%)",
        yaxis2=dict(
            title="Liquid Patrimony (R$)",
            titlefont=dict(color="black"),
            tickfont=dict(color="black"),
            anchor="x",
            overlaying="y",
            side="right"
        ),
        legend_title="Ticker",
        barmode='relative',
        hovermode="closest"
    )
    
    # Format hover info
    fig.update_traces(
        selector=dict(type='bar'),
        hovertemplate="<b>%{x}</b><br>" +
                      "Ticker: %{fullData.name}<br>" +
                      "Percentage: %{y:.2f}%<br>"
    )
    
    # Add hover template for the line
    fig.update_traces(
        selector=dict(name='Liquid Patrimony'),
        hovertemplate="<b>%{x}</b><br>" +
                      "Liquid Patrimony: R$ %{y:,.2f}<br>"
    )
    
    # Show the plot
    fig.show()
    
    return None

# Example usage:
# For a specific fund
graph_percentage_and_patrimony(df_peers, "IP")
# For a specific fund, excluding South America
graph_percentage_and_patrimony(df_peers, "IP", exclude_south_america=True)
# For all funds, BDR only, excluding South America
graph_percentage_and_patrimony(df_peers, investment_type="BDR", exclude_south_america=True)

funds,month,liquid_patrimony,investment,percentage,value,ticker,company_country,area_of_work,investment_type,date_for_sort,rentability
str,str,f64,str,f64,f64,str,str,str,str,datetime[μs],f64
"""IP""","""09/2024""",5.9431e8,"""BDR - GOGL34 DT. INI. VIGEN.: …",0.06952,4.1317e7,"""GOOGL""","""USA""","""Technology and Search Engine""","""BDR""",2024-09-01 00:00:00,0.006624
"""IP""","""09/2024""",5.9431e8,"""BDR - MSCD34 DT. INI. VIGEN.: …",0.03453,2.0522e7,"""MA""","""USA""","""Financial Services""","""BDR""",2024-09-01 00:00:00,0.006624
"""IP""","""09/2024""",5.9431e8,"""BDR - NFLX34 DT. INI. VIGEN.: …",0.02366,1.4061e7,"""NFLX""","""USA""","""Entertainment and Streaming""","""BDR""",2024-09-01 00:00:00,0.006624
"""IP""","""09/2024""",5.9431e8,"""BDR - AIRB34 DT. INI. VIGEN.: …",0.0212,1.2599e7,"""ABNB""","""USA""","""Travel and Hospitality""","""BDR""",2024-09-01 00:00:00,0.006624
"""IP""","""09/2024""",5.9431e8,"""Invest. Ext. - IP PART INTL CL""",0.22687,1.3483e8,"""Fundo""","""International""","""Investment Fund""","""Invest. Ext.""",2024-09-01 00:00:00,0.006624
…,…,…,…,…,…,…,…,…,…,…,…
"""IP""","""10/2022""",5.5978e8,"""Invest. Ext. - CHARTER COMMUNI…",0.00934,5.2284e6,"""CHTR""","""USA""","""Telecomunications""","""Invest. Ext.""",2022-10-01 00:00:00,0.054508
"""IP""","""10/2022""",5.5978e8,"""Invest. Ext. - MICROSOFT CORPO…",0.00922,5.1612e6,"""MSFT""","""USA""","""Technology and Software""","""Invest. Ext.""",2022-10-01 00:00:00,0.054508
"""IP""","""10/2022""",5.5978e8,"""Invest. Ext. - AMAZON.COM INC""",0.00583,3.2635e6,"""AMZN""","""USA""","""E-commerce and Cloud Computing""","""Invest. Ext.""",2022-10-01 00:00:00,0.054508


funds,month,liquid_patrimony,investment,percentage,value,ticker,company_country,area_of_work,investment_type,date_for_sort,rentability
str,str,f64,str,f64,f64,str,str,str,str,datetime[μs],f64
"""IP""","""09/2024""",5.9431e8,"""BDR - GOGL34 DT. INI. VIGEN.: …",0.06952,4.1317e7,"""GOOGL""","""USA""","""Technology and Search Engine""","""BDR""",2024-09-01 00:00:00,0.006624
"""IP""","""09/2024""",5.9431e8,"""BDR - MSCD34 DT. INI. VIGEN.: …",0.03453,2.0522e7,"""MA""","""USA""","""Financial Services""","""BDR""",2024-09-01 00:00:00,0.006624
"""IP""","""09/2024""",5.9431e8,"""BDR - NFLX34 DT. INI. VIGEN.: …",0.02366,1.4061e7,"""NFLX""","""USA""","""Entertainment and Streaming""","""BDR""",2024-09-01 00:00:00,0.006624
"""IP""","""09/2024""",5.9431e8,"""BDR - AIRB34 DT. INI. VIGEN.: …",0.0212,1.2599e7,"""ABNB""","""USA""","""Travel and Hospitality""","""BDR""",2024-09-01 00:00:00,0.006624
"""IP""","""09/2024""",5.9431e8,"""Invest. Ext. - IP PART INTL CL""",0.22687,1.3483e8,"""Fundo""","""International""","""Investment Fund""","""Invest. Ext.""",2024-09-01 00:00:00,0.006624
…,…,…,…,…,…,…,…,…,…,…,…
"""IP""","""10/2022""",5.5978e8,"""Invest. Ext. - CHARTER COMMUNI…",0.00934,5.2284e6,"""CHTR""","""USA""","""Telecomunications""","""Invest. Ext.""",2022-10-01 00:00:00,0.054508
"""IP""","""10/2022""",5.5978e8,"""Invest. Ext. - MICROSOFT CORPO…",0.00922,5.1612e6,"""MSFT""","""USA""","""Technology and Software""","""Invest. Ext.""",2022-10-01 00:00:00,0.054508
"""IP""","""10/2022""",5.5978e8,"""Invest. Ext. - AMAZON.COM INC""",0.00583,3.2635e6,"""AMZN""","""USA""","""E-commerce and Cloud Computing""","""Invest. Ext.""",2022-10-01 00:00:00,0.054508


C:\Users\VHENEQUIM\AppData\Local\Temp\ipykernel_33560\1766033946.py:21: UserWarning:

Comparisons with None always result in null. Consider using `.is_null()` or `.is_not_null()`.



funds,month,liquid_patrimony,investment,percentage,value,ticker,company_country,area_of_work,investment_type,date_for_sort,rentability
str,str,f64,str,f64,f64,str,str,str,str,datetime[μs],f64
"""IP""","""09/2024""",5.9431e8,"""BDR - GOGL34 DT. INI. VIGEN.: …",0.06952,4.1317e7,"""GOOGL""","""USA""","""Technology and Search Engine""","""BDR""",2024-09-01 00:00:00,0.006624
"""IP""","""09/2024""",5.9431e8,"""BDR - MSCD34 DT. INI. VIGEN.: …",0.03453,2.0522e7,"""MA""","""USA""","""Financial Services""","""BDR""",2024-09-01 00:00:00,0.006624
"""IP""","""09/2024""",5.9431e8,"""BDR - NFLX34 DT. INI. VIGEN.: …",0.02366,1.4061e7,"""NFLX""","""USA""","""Entertainment and Streaming""","""BDR""",2024-09-01 00:00:00,0.006624
"""IP""","""09/2024""",5.9431e8,"""BDR - AIRB34 DT. INI. VIGEN.: …",0.0212,1.2599e7,"""ABNB""","""USA""","""Travel and Hospitality""","""BDR""",2024-09-01 00:00:00,0.006624
"""IP""","""08/2024""",5.8901e8,"""BDR - GOGL34 DT. INI. VIGEN.: …",0.07162,4.2185e7,"""GOOGL""","""USA""","""Technology and Search Engine""","""BDR""",2024-08-01 00:00:00,0.04082
…,…,…,…,…,…,…,…,…,…,…,…
"""IP""","""10/2022""",5.5978e8,"""BDR - MSFT34 DT. INI. VIGEN.: …",0.03043,1.7034e7,"""MSFT""","""USA""","""Technology and Software""","""BDR""",2022-10-01 00:00:00,0.054508
"""IP""","""10/2022""",5.5978e8,"""BDR - LBRD34 DT. INI. VIGEN.: …",0.02931,1.6407e7,"""LBRD""","""USA""","""Telecomunications""","""BDR""",2022-10-01 00:00:00,0.054508
"""IP""","""10/2022""",5.5978e8,"""BDR - AMZO34 DT. INI. VIGEN.: …",0.02893,1.6194e7,"""AMZN""","""USA""","""E-commerce and Cloud Computing""","""BDR""",2022-10-01 00:00:00,0.054508


In [130]:

def graph_per_fund(df, fund_name, value_type="percentage", investment_type=None, exclude_south_america=False):
    """
    Generate a graph showing the investment composition of a specific fund over time.
    
    Parameters:
    -----------
    df : polars.DataFrame
        The dataframe containing the investment data
    fund_name : str
        The name of the fund to analyze
    value_type : str, default="value"
        The column to use for values ("value" for absolute or "percentage" for percentage)
    investment_type : str or None, default=None
        Filter for specific investment type ("BDR", "Invest. Ext.", or None for both)
    exclude_south_america : bool, default=False
        If True, excludes investments in companies based in South America.
    """
    # Filter for the specified fund
    filtered_df = df.filter(pl.col("funds") == fund_name)
    title_parts = [fund_name]
    
    # Apply investment type filter if specified
    if investment_type == "BDR":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "BDR")
        type_title = "BDR"
    elif investment_type == "Invest. Ext.":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "Invest. Ext.")
        type_title = "Invest. Ext."
    else:
        type_title = "BDR & Invest. Ext."
    title_parts.append(type_title)
    
    # Apply South America Filter
    if exclude_south_america:
        south_american_countries = [
            "Argentina", "Bolivia", "Brazil", "Chile", "Colombia",
            "Ecuador", "Guyana", "Paraguay", "Peru", "Suriname",
            "Uruguay", "Venezuela"
        ]
        # Ensure the country column is string type for comparison
        filtered_df = filtered_df.filter(
            ~pl.col("company_country").cast(pl.Utf8).is_in(south_american_countries)
        )
        title_parts.append("(Excluding South America)")

    # Set value type label for the title
    value_label = "Value (R$)" if value_type == "value" else "Percentage (%)"
    title_parts.append(f"({value_label})")

    graph_title = " - ".join(title_parts[:2]) # Join fund name and type
    if len(title_parts) > 3: # Add exclusion and value label parts
        graph_title += f" {title_parts[2]}"
    graph_title += f" {title_parts[-1]}" # Add value label part

    # Group by month, ticker, and date_for_sort, sum the values
    # Multiply by 100 only if value_type is percentage
    agg_expr = pl.sum(value_type)
    if value_type == "percentage":
        agg_expr = (agg_expr * 100)
        
    monthly_ticker_sum = filtered_df.group_by(["month", "ticker", "date_for_sort"]).agg(
        agg_expr.alias("total_value")
    )
    
    # Convert to pandas for easier manipulation with plotly
    plot_df = monthly_ticker_sum.to_pandas()
    
    # Check if data exists
    if plot_df.empty:
        print(f"No data found for fund: {fund_name} with specified filters.")
        return None
    
    # Sort by date to ensure chronological order
    plot_df = plot_df.sort_values('date_for_sort')
    
    # Calculate the total value per ticker across all months
    ticker_totals = plot_df.groupby('ticker')['total_value'].sum().sort_values(ascending=False)
    
    # Get ordered list of tickers for the color assignment
    ordered_tickers = ticker_totals.index.tolist()
    
    # Create stacked bar chart with plotly
    fig = px.bar(
        plot_df,
        x="month",
        y="total_value",
        color="ticker",
        title=graph_title, # Use the constructed title
        labels={
            "month": "Month",
            "total_value": value_label,
            "ticker": "Ticker"
        },
        category_orders={
            "month": plot_df["month"].tolist(),  # Preserves chronological order
            "ticker": ordered_tickers,  # Use the custom order based on total values
        },
        height=600,
    )
    
    
    # Improve layout
    fig.update_layout(
        xaxis_title="Month",
        yaxis_title=value_label,
        legend_title="Ticker",
        barmode='relative',  # Use relative instead of stack to properly handle negative values
        hovermode="closest"
    )
    
    # Format hover info
    hover_format = "%{y:.2f}%" if value_type == "percentage" else "R$ %{y:,.2f}"
    fig.update_traces(
        hovertemplate="<b>%{x}</b><br>" +
                      "Ticker: %{fullData.name}<br>" +
                      f"Value: {hover_format}<br>"
    )
    
    # Return the figure instead of showing it
    return fig

graph_per_fund(df_peers, fund_name="IP", value_type="percentage", exclude_south_america=False, investment_type="BDR").show()
graph_per_fund(df_peers, fund_name="IP", value_type="percentage", exclude_south_america=True, investment_type="BDR").show()

In [123]:
# In[10]:
def graph_per_fund(df, fund_name, value_type="percentage", investment_type=None, exclude_south_america=False):
    """
    Generate a graph showing the investment composition of a specific fund over time.
    
    Parameters:
    -----------
    df : polars.DataFrame
        The dataframe containing the investment data
    fund_name : str
        The name of the fund to analyze
    value_type : str, default="value"
        The column to use for values ("value" for absolute or "percentage" for percentage)
    investment_type : str or None, default=None
        Filter for specific investment type ("BDR", "Invest. Ext.", or None for both)
    exclude_south_america : bool, default=False
        If True, excludes investments in companies based in South America.
    """
    # Filter for the specified fund
    filtered_df = df.filter(pl.col("funds") == fund_name)
    title_parts = [fund_name]
    
    # Apply investment type filter if specified
    if investment_type == "BDR":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "BDR")
        type_title = "BDR"
    elif investment_type == "Invest. Ext.":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "Invest. Ext.")
        type_title = "Invest. Ext."
    else:
        type_title = "BDR & Invest. Ext."
    title_parts.append(type_title)
    
    # Apply South America Filter
    if exclude_south_america:
        south_american_countries = [
            "Argentina", "Bolivia", "Brazil", "Chile", "Colombia",
            "Ecuador", "Guyana", "Paraguay", "Peru", "Suriname",
            "Uruguay", "Venezuela"
        ]
        # Ensure the country column is string type for comparison
        filtered_df = filtered_df.filter(
            ~pl.col("company_country").cast(pl.Utf8).is_in(south_american_countries)
        )
        title_parts.append("(Excluding South America)")

    # Set value type label for the title
    value_label = "Value (R$)" if value_type == "value" else "Percentage (%)"
    title_parts.append(f"({value_label})")

    graph_title = " - ".join(title_parts[:2]) # Join fund name and type
    if len(title_parts) > 3: # Add exclusion and value label parts
        graph_title += f" {title_parts[2]}"
    graph_title += f" {title_parts[-1]}" # Add value label part

    # Group by month, ticker, and date_for_sort, sum the values
    # Multiply by 100 only if value_type is percentage
    agg_expr = pl.sum(value_type)
    if value_type == "percentage":
        agg_expr = (agg_expr * 100)
        
    monthly_ticker_sum = filtered_df.group_by(["month", "ticker", "date_for_sort"]).agg(
        agg_expr.alias("total_value")
    )
    
    # Convert to pandas for easier manipulation with plotly
    plot_df = monthly_ticker_sum.to_pandas()
    
    # Check if data exists
    if plot_df.empty:
        print(f"No data found for fund: {fund_name} with specified filters.")
        return None
    
    # Sort by date to ensure chronological order
    plot_df = plot_df.sort_values('date_for_sort')
    
    # Calculate the total value per ticker across all months
    ticker_totals = plot_df.groupby('ticker')['total_value'].sum().sort_values(ascending=False)
    
    # Get ordered list of tickers for the color assignment
    ordered_tickers = ticker_totals.index.tolist()
    
    # Create stacked bar chart with plotly
    fig = px.bar(
        plot_df,
        x="month",
        y="total_value",
        color="ticker",
        title=graph_title, # Use the constructed title
        labels={
            "month": "Month",
            "total_value": value_label,
            "ticker": "Ticker"
        },
        category_orders={
            "month": plot_df["month"].tolist(),  # Preserves chronological order
            "ticker": ordered_tickers,  # Use the custom order based on total values
        },
        height=600,
    )
    
    # ... existing code ...

# Example usage with the new filter:
# graph_per_fund(df_peers, fund_name="IP", value_type="percentage", exclude_south_america=True)
graph_per_fund(df_peers, fund_name="IP", value_type="percentage")


In [136]:
# In[12]:
import polars as pl
import pandas as pd
import plotly.express as px
import numpy as np

def analyze_monthly_performance_vs_percentage_for_fund(
    df,
    fund_name,
    investment_type=None,
    exclude_south_america=False
):
    """
    Analyzes the month-to-month relationship between a specific fund's performance
    (rentability) and its percentage allocation to specified investment types.

    Parameters:
    -----------
    df : polars.DataFrame
        The dataframe containing the investment data.
    fund_name : str
        The name of the fund to analyze.
    investment_type : str or None, default=None
        Filter for specific investment type ("BDR", "Invest. Ext.", or None for both).
    exclude_south_america : bool, default=False
        If True, excludes investments in companies based in South America.

    Returns:
    --------
    plotly.graph_objects.Figure or None
        A scatter plot figure visualizing the relationship, or None if insufficient data.
    """
    # --- 1. Filter Data for the Specific Fund ---
    fund_df = df.filter(pl.col("funds") == fund_name)
    if fund_df.height == 0:
        print(f"No data found for fund: {fund_name}")
        return None

    filtered_df = fund_df.clone()
    title_parts = [fund_name]

    # --- 2. Apply Investment Type and Geography Filters ---
    if investment_type == "BDR":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "BDR")
        type_title = "BDR"
    elif investment_type == "Invest. Ext.":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "Invest. Ext.")
        type_title = "Invest. Ext."
    else:
        filtered_df = filtered_df.filter(
            (pl.col("investment_type") == "BDR") | (pl.col("investment_type") == "Invest. Ext.")
        )
        type_title = "BDR & Invest. Ext."
    title_parts.append(type_title)

    if exclude_south_america:
        south_american_countries = [
            "Argentina", "Bolivia", "Brazil", "Chile", "Colombia",
            "Ecuador", "Guyana", "Paraguay", "Peru", "Suriname",
            "Uruguay", "Venezuela"
        ]
        filtered_df = filtered_df.filter(
            ~pl.col("company_country").cast(pl.Utf8).is_in(south_american_countries)
        )
        title_parts.append("(Excl. S.Am.)")

    analysis_title = " ".join(title_parts)

    # --- 3. Calculate Monthly Metrics ---
    # Group by month to get total percentage in filtered assets and the fund's rentability for that month
    monthly_metrics = filtered_df.group_by(["month", "date_for_sort"]).agg(
        (pl.sum("percentage") * 100).alias("monthly_investment_percentage"), # Sum percentages within the month
        pl.first("rentability").cast(pl.Float64).alias("monthly_rentability") # Get the unique rentability for the fund that month
    ).sort("date_for_sort").drop_nulls() # Ensure we have both metrics for the month

    # Convert to Pandas for correlation and plotting
    monthly_metrics_pd = monthly_metrics.to_pandas()

    if monthly_metrics_pd.shape[0] < 2:
        print(f"Not enough monthly data points ({monthly_metrics_pd.shape[0]}) after filtering for {analysis_title}.")
        return None

    # --- 4. Correlation Analysis ---
    # Ensure columns are numeric
    monthly_metrics_pd['monthly_investment_percentage'] = pd.to_numeric(monthly_metrics_pd['monthly_investment_percentage'], errors='coerce')
    monthly_metrics_pd['monthly_rentability'] = pd.to_numeric(monthly_metrics_pd['monthly_rentability'], errors='coerce')
    monthly_metrics_pd = monthly_metrics_pd.dropna(subset=['monthly_investment_percentage', 'monthly_rentability'])

    if monthly_metrics_pd.shape[0] < 2:
        print(f"Not enough numeric monthly data points ({monthly_metrics_pd.shape[0]}) after cleaning for {analysis_title}.")
        return None

    correlation = monthly_metrics_pd['monthly_investment_percentage'].corr(monthly_metrics_pd['monthly_rentability'])

    print(f"Monthly Analysis for Fund: {analysis_title}")
    print(f"Correlation between Monthly Investment Percentage and Monthly Rentability: {correlation:.4f}")

    # --- 5. Visualization ---
    plot_title = f'{fund_name}: Monthly Rentability vs. Investment Pct ({type_title})'
    if exclude_south_america:
        plot_title += " (Excl. S.Am.)"
    plot_title += f'<br>Correlation: {correlation:.4f}'

    fig = px.scatter(
        monthly_metrics_pd,
        x="monthly_investment_percentage",
        y="monthly_rentability",
        title=plot_title,
        labels={
            "monthly_investment_percentage": f"Monthly Investment Percentage ({type_title}) (%)",
            "monthly_rentability": "Monthly Rentability (%)"
        },
        hover_name="month", # Show month on hover
        trendline="ols",
        trendline_color_override="red"
    )

    fig.update_traces(
        marker=dict(size=10),
        hovertemplate="<b>Month: %{hovertext}</b><br>" +
                      "Investment Pct: %{x:.2f}%<br>" +
                      "Rentability: %{y:.4f}%<extra></extra>"
    )

    fig.update_layout(height=600)

    return fig

# --- Example Usage ---
# Analyze IP fund, BDR & Invest. Ext., including South America
fig_ip_all = analyze_monthly_performance_vs_percentage_for_fund(df_peers, "IP")
if fig_ip_all: fig_ip_all.show()

# Analyze IP fund, BDR only, excluding South America
fig_ip_bdr_no_sa = analyze_monthly_performance_vs_percentage_for_fund(
    df_peers,
    "IP",
    investment_type="BDR",
    exclude_south_america=True
)
if fig_ip_bdr_no_sa: fig_ip_bdr_no_sa.show()

# Analyze VINLAND fund, Invest. Ext. only, including South America
# fig_vin_ext = analyze_monthly_performance_vs_percentage_for_fund(
#     df_peers,
#     "VINLAND",
#     investment_type="Invest. Ext."
# )
# if fig_vin_ext: fig_vin_ext.show()

Monthly Analysis for Fund: IP BDR & Invest. Ext.
Correlation between Monthly Investment Percentage and Monthly Rentability: -0.3242


Monthly Analysis for Fund: IP BDR (Excl. S.Am.)
Correlation between Monthly Investment Percentage and Monthly Rentability: -0.3288


In [144]:
# In[9]:
import polars as pl
import pandas as pd
import plotly.express as px
import numpy as np

def analyze_performance_vs_investment_percentage(
    df,
    investment_type=None,
    exclude_south_america=False,
    top_n_funds=None # Add parameter to select top N funds
):
    """
    Analyzes the correlation between average fund performance (rentability)
    and the average percentage allocated to specified investment types,
    with an option to exclude South American companies and focus on top performing funds.

    Parameters:
    -----------
    df : polars.DataFrame
        The dataframe containing the investment data, including 'funds',
        'month', 'percentage', 'rentability', 'investment_type', and 'company_country' columns.
    investment_type : str or None, default=None
        Filter for specific investment type:
        - "BDR": Only include BDR investments.
        - "Invest. Ext.": Only include Invest. Ext. investments.
        - None (default): Include both BDR and Invest. Ext. investments.
    exclude_south_america : bool, default=False
        If True, excludes investments in companies based in South America.
    top_n_funds : int or None, default=None
        If an integer is provided, analyze only the top N funds based on average rentability.
        If None, analyze all eligible funds.
    """
    # --- 1. Filter Data ---
    filtered_df = df.clone()
    title_parts = []

    # Apply Investment Type Filter
    if investment_type == "BDR":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "BDR")
        type_title = "BDR"
    elif investment_type == "Invest. Ext.":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "Invest. Ext.")
        type_title = "Invest. Ext."
    else:
        # Use both BDR and Invest. Ext. if None is specified
        filtered_df = filtered_df.filter(
            (pl.col("investment_type") == "BDR") | (pl.col("investment_type") == "Invest. Ext.")
        )
        type_title = "BDR & Invest. Ext."
    title_parts.append(type_title)

    # Apply South America Filter
    if exclude_south_america:
        south_american_countries = [
            "Argentina", "Bolivia", "Brazil", "Chile", "Colombia",
            "Ecuador", "Guyana", "Paraguay", "Peru", "Suriname",
            "Uruguay", "Venezuela"
            # Add French Guiana if needed, though it's technically an overseas department of France
        ]
        # Ensure the country column is string type for comparison
        filtered_df = filtered_df.filter(
            ~pl.col("company_country").cast(pl.Utf8).is_in(south_american_countries)
        )
        title_parts.append("(Excluding South America)")

    analysis_title = " ".join(title_parts)

    # --- 2. Calculate Monthly Metrics per Fund ---
    # Group by fund and month to get total percentage and unique rentability
    # Ensure rentability is numeric before aggregation
    monthly_fund_metrics = filtered_df.group_by(["funds", "month"]).agg(
        pl.sum("percentage").alias("total_monthly_percentage"),
        pl.first("rentability").cast(pl.Float64).alias("monthly_rentability") # Cast to float
    )

    # --- 3. Aggregate Data per Fund ---
    # Calculate average percentage and average rentability for each fund over time
    fund_summary = monthly_fund_metrics.group_by("funds").agg(
        (pl.mean("total_monthly_percentage")*100).alias("avg_investment_percentage"),
        (pl.mean("monthly_rentability")*100).alias("avg_rentability") # Multiply rentability by 100 for percentage display
    ).drop_nulls() # Remove funds with missing data for either metric

    # --- 3.5 Filter for Top N Funds (if specified) ---
    if top_n_funds is not None and isinstance(top_n_funds, int) and top_n_funds > 0:
        if fund_summary.height > top_n_funds:
            fund_summary = fund_summary.sort("avg_rentability", descending=True).head(top_n_funds)
            title_parts.append(f"(Top {top_n_funds} Funds)")
            analysis_title = " ".join(title_parts) # Update title if top N is applied

    # Convert to Pandas for correlation and plotting
    fund_summary_pd = fund_summary.to_pandas()

    if fund_summary_pd.shape[0] < 2:
        print(f"Not enough data points ({fund_summary_pd.shape[0]}) to calculate correlation for {analysis_title}.")
        return None # Return only one value as per original structure

    # --- 4. Correlation Analysis ---
    # Ensure columns are numeric before calculating correlation
    fund_summary_pd['avg_investment_percentage'] = pd.to_numeric(fund_summary_pd['avg_investment_percentage'], errors='coerce')
    fund_summary_pd['avg_rentability'] = pd.to_numeric(fund_summary_pd['avg_rentability'], errors='coerce')
    fund_summary_pd = fund_summary_pd.dropna(subset=['avg_investment_percentage', 'avg_rentability'])

    if fund_summary_pd.shape[0] < 2:
        print(f"Not enough numeric data points ({fund_summary_pd.shape[0]}) after cleaning for {analysis_title}.")
        return None # Return only one value

    correlation = fund_summary_pd['avg_investment_percentage'].corr(fund_summary_pd['avg_rentability'])

    print(f"Analysis for Investment Type: {analysis_title}")
    print(f"Correlation between Average Investment Percentage and Average Rentability: {correlation:.4f}")

    # --- 5. Visualization ---
    plot_title = f'Fund Performance vs. Avg. Investment Percentage ({analysis_title})<br>Correlation: {correlation:.4f}'
    fig = px.scatter(
        fund_summary_pd,
        x="avg_investment_percentage",
        y="avg_rentability",
        title=plot_title,
        labels={
            "avg_investment_percentage": f"Average Monthly Investment Percentage ({type_title}) (%)",
            "avg_rentability": "Average Monthly Rentability (%)"
        },
        hover_name="funds", # Show fund name on hover
        trendline="ols", # Add a linear regression trendline
        trendline_color_override="red"
    )

    fig.update_traces(
        marker=dict(size=10),
        hovertemplate="<b>%{hovertext}</b><br>" +
                      "Avg. Investment Pct: %{x:.2f}%<br>" +
                      "Avg. Rentability: %{y:.2f}%<extra></extra>" # Update format for percentage rentability
    )

    fig.update_layout(height=600)

    return fig

# --- Example Usage ---
# Make sure df_peers is loaded and contains the necessary columns

# Analyze for BDR & Invest. Ext. combined (default), including South America (default)
# fig_both_all = analyze_performance_vs_investment_percentage(df_peers)
# if fig_both_all: fig_both_all.show()

# Analyze for BDR only, including South America
# fig_bdr_all = analyze_performance_vs_investment_percentage(df_peers, investment_type="BDR")
# if fig_bdr_all: fig_bdr_all.show()

# Analyze for Invest. Ext. only, excluding South America
# fig_ext_no_sa = analyze_performance_vs_investment_percentage(df_peers, investment_type="Invest. Ext.", exclude_south_america=True)
# if fig_ext_no_sa: fig_ext_no_sa.show()

# Analyze for BDR & Invest. Ext. combined, excluding South America
# fig_both_no_sa = analyze_performance_vs_investment_percentage(df_peers, exclude_south_america=True)
# if fig_both_no_sa: fig_both_no_sa.show()

# Analyze for BDR & Invest. Ext. combined, excluding South America, Top 10 Funds
fig_top10 = analyze_performance_vs_investment_percentage(
    df_peers,
    exclude_south_america=True,
    top_n_funds=10
)
if fig_top10: fig_top10.show()

# Analyze for BDR only, including South America, Top 5 Funds
fig_top5_bdr = analyze_performance_vs_investment_percentage(
    df_peers,
    exclude_south_america=True,
    top_n_funds=5
)
if fig_top5_bdr: fig_top5_bdr.show()

Analysis for Investment Type: BDR & Invest. Ext. (Excluding South America) (Top 10 Funds)
Correlation between Average Investment Percentage and Average Rentability: -0.3581


Analysis for Investment Type: BDR & Invest. Ext. (Excluding South America) (Top 5 Funds)
Correlation between Average Investment Percentage and Average Rentability: -0.2153


In [148]:
# In[9]:
import polars as pl
import pandas as pd
import plotly.express as px
import numpy as np

def analyze_performance_vs_investment_percentage(
    df,
    investment_type=None,
    exclude_south_america=False,
    top_n_funds=None,
    num_extremes=None # Add parameter for top/bottom N funds
):
    """
    Analyzes the correlation between average fund performance (rentability)
    and the average percentage allocated to specified investment types.

    Allows filtering by investment type, excluding South American companies,
    focusing on the top N performing funds, or focusing on the top and bottom N funds.

    Parameters:
    -----------
    df : polars.DataFrame
        The dataframe containing the investment data.
    investment_type : str or None, default=None
        Filter for specific investment type ("BDR", "Invest. Ext.", or None for both).
    exclude_south_america : bool, default=False
        If True, excludes investments in companies based in South America.
    top_n_funds : int or None, default=None
        If an integer is provided, analyze only the top N funds based on average rentability.
        Mutually exclusive with num_extremes.
    num_extremes : int or None, default=None
        If an integer is provided, analyze the top N and bottom N funds based on average rentability.
        Mutually exclusive with top_n_funds. If None, analyze all eligible funds (or top_n_funds if specified).
    """
    # --- Parameter Validation ---
    if top_n_funds is not None and num_extremes is not None:
        raise ValueError("Parameters 'top_n_funds' and 'num_extremes' are mutually exclusive. Please provide only one.")

    # --- 1. Filter Data ---
    filtered_df = df.clone()
    title_parts = []

    # Apply Investment Type Filter
    if investment_type == "BDR":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "BDR")
        type_title = "BDR"
    elif investment_type == "Invest. Ext.":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "Invest. Ext.")
        type_title = "Invest. Ext."
    else:
        filtered_df = filtered_df.filter(
            (pl.col("investment_type") == "BDR") | (pl.col("investment_type") == "Invest. Ext.")
        )
        type_title = "BDR & Invest. Ext."
    title_parts.append(type_title)

    # Apply South America Filter
    if exclude_south_america:
        south_american_countries = [
            "Argentina", "Bolivia", "Brazil", "Chile", "Colombia",
            "Ecuador", "Guyana", "Paraguay", "Peru", "Suriname",
            "Uruguay", "Venezuela"
        ]
        filtered_df = filtered_df.filter(
            ~pl.col("company_country").cast(pl.Utf8).is_in(south_american_countries)
        )
        title_parts.append("(Excluding South America)")

    analysis_title_base = " ".join(title_parts) # Base title before fund selection

    # --- 2. Calculate Monthly Metrics per Fund ---
    monthly_fund_metrics = filtered_df.group_by(["funds", "month"]).agg(
        pl.sum("percentage").alias("total_monthly_percentage"),
        pl.first("rentability").cast(pl.Float64).alias("monthly_rentability")
    )

    # --- 3. Aggregate Data per Fund ---
    fund_summary = monthly_fund_metrics.group_by("funds").agg(
        (pl.mean("total_monthly_percentage")*100).alias("avg_investment_percentage"),
        (pl.mean("monthly_rentability")*100).alias("avg_rentability")
    ).drop_nulls()

    # --- 3.5 Filter Funds based on Performance ---
    analysis_title = analysis_title_base # Start with base title
    if num_extremes is not None and isinstance(num_extremes, int) and num_extremes > 0:
        if fund_summary.height >= 2 * num_extremes:
            fund_summary_sorted = fund_summary.sort("avg_rentability")
            top_funds = fund_summary_sorted.tail(num_extremes) # Highest rentability
            bottom_funds = fund_summary_sorted.head(num_extremes) # Lowest rentability
            fund_summary = pl.concat([top_funds, bottom_funds])
            title_parts.append(f"(Top & Bottom {num_extremes} Funds)")
            analysis_title = " ".join(title_parts)
        else:
            print(f"Warning: Not enough funds ({fund_summary.height}) to select top and bottom {num_extremes}. Analyzing all available funds.")
            analysis_title = analysis_title_base + " (All Funds)" # Revert title addition

    elif top_n_funds is not None and isinstance(top_n_funds, int) and top_n_funds > 0:
        if fund_summary.height > top_n_funds:
            fund_summary = fund_summary.sort("avg_rentability", descending=True).head(top_n_funds)
            title_parts.append(f"(Top {top_n_funds} Funds)")
            analysis_title = " ".join(title_parts)
        else:
             print(f"Warning: Not enough funds ({fund_summary.height}) to select top {top_n_funds}. Analyzing all available funds.")
             analysis_title = analysis_title_base + " (All Funds)" # Revert title addition
    else:
         analysis_title = analysis_title_base + " (All Funds)" # Explicitly mention all funds if no filter

    # Convert to Pandas for correlation and plotting
    fund_summary_pd = fund_summary.to_pandas()

    if fund_summary_pd.shape[0] < 2:
        print(f"Not enough data points ({fund_summary_pd.shape[0]}) to calculate correlation for {analysis_title}.")
        return None

    # --- 4. Correlation Analysis ---
    fund_summary_pd['avg_investment_percentage'] = pd.to_numeric(fund_summary_pd['avg_investment_percentage'], errors='coerce')
    fund_summary_pd['avg_rentability'] = pd.to_numeric(fund_summary_pd['avg_rentability'], errors='coerce')
    fund_summary_pd = fund_summary_pd.dropna(subset=['avg_investment_percentage', 'avg_rentability'])

    if fund_summary_pd.shape[0] < 2:
        print(f"Not enough numeric data points ({fund_summary_pd.shape[0]}) after cleaning for {analysis_title}.")
        return None

    correlation = fund_summary_pd['avg_investment_percentage'].corr(fund_summary_pd['avg_rentability'])

    print(f"Analysis for: {analysis_title}")
    print(f"Correlation between Average Investment Percentage and Average Rentability: {correlation:.4f}")

    # --- 5. Visualization ---
    plot_title = f'Fund Performance vs. Avg. Investment Percentage ({analysis_title})<br>Correlation: {correlation:.4f}'
    fig = px.scatter(
        fund_summary_pd,
        x="avg_investment_percentage",
        y="avg_rentability",
        title=plot_title,
        labels={
            "avg_investment_percentage": f"Average Monthly Investment Percentage ({type_title}) (%)",
            "avg_rentability": "Average Monthly Rentability (%)"
        },
        hover_name="funds",
        trendline="ols",
        trendline_color_override="red"
    )

    fig.update_traces(
        marker=dict(size=10),
        hovertemplate="<b>%{hovertext}</b><br>" +
                      "Avg. Investment Pct: %{x:.2f}%<br>" +
                      "Avg. Rentability: %{y:.2f}%<extra></extra>"
    )

    fig.update_layout(height=600)

    return fig

# --- Example Usage ---

# Analyze for BDR & Invest. Ext. combined, excluding South America, Top 10 & Bottom 10 Funds
fig_extremes = analyze_performance_vs_investment_percentage(
    df_peers,
    exclude_south_america=True,
    num_extremes=10
)
if fig_extremes: fig_extremes.show()

# Analyze for BDR only, including South America, Top 5 & Bottom 5 Funds
fig_extremes_bdr = analyze_performance_vs_investment_percentage(
    df_peers.filter(pl.col("funds") != "OCEANA"),
    investment_type="BDR",
    num_extremes=5
)
if fig_extremes_bdr: fig_extremes_bdr.show()

# Analyze for BDR & Invest. Ext. combined, excluding South America, Top 10 Funds (using the other parameter)
# fig_top10 = analyze_performance_vs_investment_percentage(
#     df_peers,
#     exclude_south_america=True,
#     top_n_funds=10
# )
# if fig_top10: fig_top10.show()

# Analyze all funds (default)
# fig_all = analyze_performance_vs_investment_percentage(df_peers)
# if fig_all: fig_all.show()

Analysis for: BDR & Invest. Ext. (Excluding South America) (Top & Bottom 10 Funds)
Correlation between Average Investment Percentage and Average Rentability: 0.1972


Analysis for: BDR (Top & Bottom 5 Funds)
Correlation between Average Investment Percentage and Average Rentability: 0.4752
